<a href="https://colab.research.google.com/github/aadicodes/LLM-Chat/blob/main/SQLite_Mistral_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers sentence-transformers gradio datasets accelerate

In [ ]:
from google.colab import files

#enterprise_data.db
import pandas as pd
import sqlite3


conn = sqlite3.connect("enterprise_data.db")
cursor = conn.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

print("Tables in database:", tables)

df_new = pd.DataFrame([
    {'branch_name': 'Miami Branch', 'revenue': 72000.00, 'quarter': 'Q2', 'year':2025},
    {'branch_name': 'Dallas Branch', 'revenue': 88000.00, 'quarter': 'Q2', 'year':2025},
    {'branch_name': 'Phoenix Branch', 'revenue': 188000.00, 'quarter': 'Q3', 'year':2025},
    {'branch_name': 'Austin Branch', 'revenue': 128000.00, 'quarter': 'Q3', 'year':2025},
    {'branch_name': 'Denver Branch', 'revenue': 126000.00, 'quarter': 'Q3', 'year':2025}
])

df_new.to_sql('branch_revenue', conn, if_exists='append', index=False)


In [ ]:

from sentence_transformers import SentenceTransformer, util

# Connect to uploaded SQLite file
conn = sqlite3.connect("enterprise_data.db")  # Replace with your file name
cursor = conn.cursor()

# Load table data
df = pd.read_sql_query("SELECT * FROM branch_revenue", conn)
print(df)
conn.close()

# Embed rows
embedder = SentenceTransformer("all-MiniLM-L6-v2")
corpus = df.astype(str).agg(" ".join, axis=1).tolist()
corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"  # No token required
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)
chatbot = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [ ]:
def answer_question(user_query):
    query_embedding = embedder.encode(user_query, convert_to_tensor=True)
    hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=3)[0]
    context = "\n".join([corpus[hit['corpus_id']] for hit in hits])

    prompt = f"""You are a helpful assistant. Use the following context to answer the question.

Context:
{context}

Question:
{user_query}

Answer:"""

    response = chatbot(prompt, max_new_tokens=200, do_sample=True)[0]['generated_text']
    return response.split("Answer:")[-1].strip()

In [ ]:
import gradio as gr

gr.Interface(fn=answer_question, inputs="text", outputs="text",
             title="📊 Chatbot using Mistral LLM",
             description="Ask questions about your enterprise data stored in SQLite.").launch()
